## Image Colorization using U-Net and GANs
#### 1. Importing Necessary Libraries

## Step 1. Importing necessary libraries and Setting Device

In [ ]:
import torch
import torchvision
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from torch import nn, optim

import numpy as np
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.gridspec as gs
from matplotlib import font_manager as fm, rcParams
from PIL import Image
from skimage.color import rgb2lab, lab2rgb

from tqdm import tqdm

from datetime import datetime

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

## Step 2. Fetching The Dataset and Setting Up Input Paths

### Installing fastai for Quickly Getting The COCO Dataset

In [ ]:
!pip install -U fastai
import fastai

### Grabbing The Dataset in the Following Directory Structure
<pre>
.
└── .fastai
    └── data
        └── coco_sample
            └── train_sample
                └── *.jpg (10,000 images in total)</pre>

In [ ]:
from fastai.data.external import untar_data, URLs
coco_path = untar_data(URLs.COCO_SAMPLE)
coco_path = str(coco_path) + "/train_sample"
paths = glob.glob(coco_path + "/*.jpg")
# Setting seed for getting the same data across all train sessions 
np.random.seed(123)
paths_subset = np.random.choice(paths, 10_000, replace=False) # choosing 10000 images randomly
rand_idxs = np.random.permutation(10_000)
train_idxs = rand_idxs[:8000] # choosing the first 8000 as training set
val_idxs = rand_idxs[8000:] # choosing last 2000 as validation set
train_paths = paths_subset[train_idxs]
val_paths = paths_subset[val_idxs]
print(train_paths)

### Previewing The Input Images

In [ ]:
imageCount = 0
fig, ax = plt.subplots(4, 4, figsize=(13,13))
for i in range(4):
    for j in range(4):    
        ax[i, j].imshow(Image.open(train_paths[imageCount]))
        imageCount+=1